## License 

Copyright 2019 Patrick Hall and the H2O.ai team

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

**DISCLAIMER:** This notebook is not legal compliance advice.

# All models are wrong, but ... why is my model wrong? 
**Part 2: Residual analysis for model debugging**

"All models are wrong but some are useful" -- George Box, 1978

This notebook uses variants of residual analysis to find error mechanisms and security vulnerabilities and to assess stability and fairness in a trained XGBoost model. It begins by loading the UCI credit card default data and then training an interpretable, monotonically constrained XGBoost gradient boosting machine (GBM) model. (Pearson correlation with the prediction target is used to determine the direction of the monotonicity constraints for each input variable.) After the model is trained, its logloss residuals are analyzed and explained thoroughly and the constrained GBM is compared to a benchmark linear model. These model debugging exercises uncover several accuracy, drift, and security problems such as over-emphasis of important variables and strong signal in model residuals. Several remediation mechanisms are proposed including missing value injection during training, additional data collection, and use of assertions to correct known problems during scoring. 

#### Start H2O cluster

The `os` commands below check whether this notebook is being run on the Aquarium platform. 

In [ ]:
import os

startup = '/home/h2o/bin/aquarium_startup'
if os.path.exists(startup):
    os.system(startup)
    local_url = 'http://localhost:54321/h2o'
    aquarium = True
    !sleep 5
else:
    local_url = 'http://localhost:54321'
    aquarium = False

#### Python imports
In general, NumPy and Pandas will be used for data manipulation purposes and h2o will be used for modeling tasks. 

In [ ]:
import numpy as np                   # array, vector, matrix calculations
import pandas as pd                  # DataFrame handling
import shap                          # for consistent, signed variable importance measurements
import xgboost as xgb                # gradient boosting machines (GBMs)
import subprocess                    # manage external processes

import h2o                                                        # import h2o python bindings to java server
from h2o.backend import H2OLocalServer                            # for plotting local tree in-notebook
from h2o.estimators.random_forest import H2ORandomForestEstimator # for single tree
from h2o.estimators.glm import H2OGeneralizedLinearEstimator      # for benchmark model
from h2o.grid.grid_search import H2OGridSearch                    # for benchmark model

# plotting ###########
import matplotlib.pyplot as plt                                   # general plotting
from matplotlib.lines import Line2D                               # necessary for custom legends
import seaborn as sns                                             # facet grid for residuals
from mpl_toolkits import mplot3d                                  # 3-D scatterplots
import matplotlib; matplotlib.rcParams.update({'font.size': 40})  # set legible font size

# suppress irrelevant warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# enables display of plots in notebook
# in-notebook display
from IPython.display import Image
from IPython.display import display
%matplotlib inline

pd.options.display.max_columns = 999 # enable display of all dataframe columns in notebook

SEED = 12345                         # global random seed
np.random.seed(SEED)                 # set random seed for reproducibility

#### Start h2o
H2o is both a library and a server. The machine learning algorithms in the library take advantage of the multithreaded and distributed architecture provided by the server to train machine learning algorithms extremely efficiently. The API for the library was imported above in cell 2, but the server still needs to be started.

>The parameters used in `h2o.init` will depend on your specific environment. Regardless of how H2O is installed, if you start a cluster, you will need to ensure that it is shut down when you are done.

In [ ]:
h2o.init(url=local_url, max_mem_size='2G')
h2o.remove_all()    # remove any existing data structures from h2o memory

## 1. Download, explore, and prepare UCI credit card default data

UCI credit card default data: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

The UCI credit card default data contains demographic and payment information about credit card customers in Taiwan in the year 2005. The data set contains 23 input variables: 

* **`LIMIT_BAL`**: Amount of given credit (NT dollar)
* **`SEX`**: 1 = male; 2 = female
* **`EDUCATION`**: 1 = graduate school; 2 = university; 3 = high school; 4 = others 
* **`MARRIAGE`**: 1 = married; 2 = single; 3 = others
* **`AGE`**: Age in years 
* **`PAY_0`, `PAY_2` - `PAY_6`**: History of past payment; `PAY_0` = the repayment status in September, 2005; `PAY_2` = the repayment status in August, 2005; ...; `PAY_6` = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; ...; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
* **`BILL_AMT1` - `BILL_AMT6`**: Amount of bill statement (NT dollar). `BILL_AMNT1` = amount of bill statement in September, 2005; `BILL_AMT2` = amount of bill statement in August, 2005; ...; `BILL_AMT6` = amount of bill statement in April, 2005. 
* **`PAY_AMT1` - `PAY_AMT6`**: Amount of previous payment (NT dollar). `PAY_AMT1` = amount paid in September, 2005; `PAY_AMT2` = amount paid in August, 2005; ...; `PAY_AMT6` = amount paid in April, 2005. 

These 23 input variables are used to predict the target variable, whether or not a customer defaulted on their credit card bill in late 2005. Because XGBoost accepts only numeric inputs, all variables will be treated as numeric.

#### Import data and clean
The credit card default data is available as an `.xls` file. Pandas reads `.xls` files automatically, so it's used to load the credit card default data and give the prediction target a shorter name: `DEFAULT_NEXT_MONTH`. 

In [ ]:
# import XLS file
path = 'default_of_credit_card_clients.xls'
data = pd.read_excel(path,
                     skiprows=1) # skip the first row of the spreadsheet

# remove spaces from target column name 
data = data.rename(columns={'default payment next month': 'DEFAULT_NEXT_MONTH'}) 

#### Assign modeling roles

The shorthand name `y` is assigned to the prediction target. `X` is assigned to all other input variables in the credit card default data except the row indentifier, `ID`.

In [ ]:
# assign target and inputs for GBM
y = 'DEFAULT_NEXT_MONTH'
X = [name for name in data.columns if name not in [y, 'ID']]
print('y =', y)
print('X =', X)

#### Display descriptive statistics
The Pandas `describe()` function displays a brief description of the credit card default data. The input variables `SEX`, `EDUCATION`, `MARRIAGE`, `PAY_0`-`PAY_6`, and the prediction target `DEFAULT_NEXT_MONTH`, are really categorical variables, but they have already been encoded into meaningful numeric, integer values, which is great for XGBoost. Also, there are no missing values in this dataset.

In [ ]:
data[X + [y]].describe() # display descriptive statistics for all columns

## 2. Investigate pair-wise Pearson correlations for DEFAULT_NEXT_MONTH

Monotonic relationships are much easier to explain to colleagues, bosses, customers, and regulators than more complex, non-monotonic relationships and monotonic relationships may also prevent overfitting and excess error due to variance for new data.

To train a transparent monotonic classifier, contraints must be supplied to XGBoost that determine whether the learned relationship between an input variable and the prediction target `DEFAULT_NEXT_MONTH` will be increasing for increases in an input variable or decreasing for increases in an input variable. Pearson correlation provides a linear measure of the direction of the relationship between each input variable and the target. If the pair-wise Pearson correlation between an input and `DEFAULT_NEXT_MONTH` is positive, it will be constrained to have an increasing relationship with the predictions for `DEFAULT_NEXT_MONTH`. If the pair-wise Pearson correlation is negative, the input will be constrained to have a decreasing relationship with the predictions for `DEFAULT_NEXT_MONTH`. 

Constrainsts are supplied to XGBoost in the form of a Python tuple with length equal to the number of inputs. Each item in the tuple is associated with an input variable based on its index in the tuple. The first constraint in the tuple is associated with the first variable in the training data, the second constraint in the tuple is associated with the second variable in the training data, and so on. The constraints themselves take the form of a 1 for a positive relationship and a -1 for a negative relationship.

#### Calculate Pearson correlation

The Pandas `.corr()` function returns the pair-wise Pearson correlation between variables in a Pandas DataFrame. Because `DEFAULT_NEXT_MONTH` is the last column in the `data` DataFrame, the last column of the Pearson correlation matrix indicates the direction of the linear relationship between each input variable and the prediction target, `DEFAULT_NEXT_MONTH`. According to the calculated values, as a customer's balance limit (`LIMIT_BAL`), bill amounts (`BILL_AMT1`-`BILL_AMT6`), and payment amounts (`PAY_AMT1`-`PAY_AMT6`) increase, their probability of default tends to decrease. However as a customer's number of late payments increase (`PAY_0`, `PAY_2`-`PAY6`), their probability of default usually increases. In general, the Pearson correlation values make sense, and they will be used to ensure that the modeled relationships will make sense as well. (Pearson correlation values between the target variable, DEFAULT_NEXT_MONTH, and each input variable are displayed directly below.)

In [ ]:
# displays last column of Pearson correlation matrix as Pandas DataFrame
pd.DataFrame(data[X + [y]].corr()[y]).iloc[:-1] 

#### Create tuple of monotonicity constraints from Pearson correlation values

The last column of the Pearson correlation matrix is transformed from a numeric column in a Pandas DataFrame into a Python tuple of `1`s and `-1`s that will be used to specifiy monotonicity constraints for each input variable in XGBoost. If the Pearson correlation between an input variable and `DEFAULT_NEXT_MONTH` is positive, a positive montonic relationship constraint is specified for that variable using `1`. If the correlation is negative, a negative monotonic constraint is specified using `-1`. (Specifying `0` indicates that no constraints should be used.) The resulting tuple will be passed to XGBoost when the GBM model is trained.

In [ ]:
# creates a tuple in which positive correlation values are assigned a 1
# and negative correlation values are assigned a -1
mono_constraints = tuple([int(i) for i in np.sign(data[X + [y]].corr()[y].values[:-1])])

# (-1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1)

## 3. Train XGBoost with monotonicity constraints

XGBoost is a very accurate, open source GBM library for regression and classification tasks. XGBoost can learn complex relationships between input variables and a target variable, but here the `monotone_constraints` tuning parameter is used to enforce monotonicity between inputs and the prediction for `DEFAULT_NEXT_MONTH`.

XGBoost is available from: https://github.com/dmlc/xgboost and the implementation of XGBoost is described in detail here: http://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf.

#### Split data into training and test sets for early stopping

The credit card default data is split into training and test sets to monitor and prevent overtraining. Reproducibility is another important factor in creating trustworthy models, and randomly splitting datasets can introduce randomness in model predictions and other results. A random seed is used here to ensure the data split is reproducible.

In [ ]:
split_ratio = 0.70    # 70%/30% train/test split

# execute split
split = np.random.rand(len(data)) < split_ratio
train = data[split]
test = data[~split]

# summarize split
print('Train data rows = %d, columns = %d' % (train.shape[0], train.shape[1]))
print('Test data rows = %d, columns = %d' % (test.shape[0], test.shape[1]))

#### Train XGBoost GBM classifier
To train an XGBoost classifier, the training and test data must be converted from Pandas DataFrames into SVMLight format. The `DMatrix()` function in the XGBoost package is used to convert the data. Many XGBoost tuning parameters must be specified as well. Typically a grid search would be performed to identify the best parameters for a given modeling task. For brevity's sake, a previously-discovered set of good tuning parameters are specified here. Notice that the monotonicity constraints are passed to XGBoost using the `monotone_constraints` parameter. Because gradient boosting methods typically resample training data, an additional random seed is also specified for XGBoost using the `seed` paramter to create reproducible predictions, error rates, and variable importance values. To calculate Shapley contributions to logloss in later sections of the notebook, early stopping is not used because it may be unsupported for calculating accurate contributions to logloss. Hence, a previously discovered best number of trees is used in this notebook.

In [ ]:
# XGBoost uses SVMLight data structure, not Numpy arrays or Pandas DataFrames 
dtrain = xgb.DMatrix(train[X], train[y])
dtest = xgb.DMatrix(test[X], test[y])

# used to calibrate predictions to mean of y 
base_y = train[y].mean()

# tuning parameters
params = {
    'objective': 'binary:logistic',             # produces 0-1 probabilities for binary classification
    'booster': 'gbtree',                        # base learner will be decision tree
    'eval_metric': 'logloss',                   # stop training based on
    'eta': 0.08,                                # learning rate
    'subsample': 0.9,                           # use 90% of rows in each decision tree
    'colsample_bytree': 0.9,                    # use 90% of columns in each decision tree
    'max_depth': 15,                            # allow decision trees to grow to depth of 15
    'monotone_constraints':mono_constraints,    # 1 = increasing relationship, -1 = decreasing relationship
    'base_score': base_y,                       # calibrate predictions to mean of y 
    'seed': SEED                                # set random seed for reproducibility
}

# watchlist is used for early stopping
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

# train model
xgb_model = xgb.train(params,                   # set tuning parameters from above                   
                      dtrain,                   # training data
                      62,                       # **logloss shap may not account for early stopping**
                      evals=watchlist,          # use watchlist for early stopping 
                      verbose_eval=True)        # display iteration progress


## 4. Cutoff Selection
Cutoffs affect model characteristics, and sometimes drastically. Cutoffs should always be chosen with care. In this notebook, the cutoff is selected by maximizing the model's F1 statistic. This is a standard approach that could likely be improved upon.

#### Preliminaries to calculate PR-AUC
A new column will be created called `p_DEFAULT_NEXT_MONTH` -- this is the model's predicted probability. P-R AUC stands for precision-recall area under the curve. The curve to be created is an established model assessment technique that helps choose a probability cutoff.

In [ ]:
# shortcut name
yhat = 'p_DEFAULT_NEXT_MONTH' 

# copy test data and reset index
test_yhat = test.copy(deep=True)
test_yhat.reset_index(drop=True, inplace=True) # Pandas joins are weird otherwise
test_yhat[yhat] = pd.DataFrame(xgb_model.predict(xgb.DMatrix(test[X]))) 

#### Function to calculate precision and recall 
Precision refers to the proportion of people the model predicts will default correctly *out of the total number of predictions of default*. Recall refers to the proportion of people the model predicts will default correctly *out of the total number of actual defaults*. F1 is a combination (harmonic mean) of these quantities. So by maximizing F1, a cutoff that considers true positives carefully is selected. The function below calculates precision, recall, and F1 at a number of probability cutoffs between 0 and 1. Results are displayed below.

In [ ]:
def get_prauc(frame, y, yhat, pos=1, neg=0, res=0.01):
    
    """ Calculates precision, recall, and f1 for a pandas dataframe of y 
        and yhat values.
    
    Args:
        frame: Pandas dataframe of actual (y) and predicted (yhat) values.
        y: Name of actual value column.
        yhat: Name of predicted value column.
        pos: Primary target value, default 1.
        neg: Secondary target value, default 0.
        res: Resolution by which to loop through cutoffs, default 0.01.
    
    Returns:
        Pandas dataframe of precision, recall, and f1 values.
        
    """
    
    frame_ = frame.copy(deep=True) # don't destroy original data
    dname = 'd_' + str(y)          # column for predicted decisions
    eps = 1e-20                    # for safe numerical operations
    
    # init p-r roc frame
    prroc_frame = pd.DataFrame(columns=['cutoff', 'recall', 'precision', 'f1'])
    
    # loop through cutoffs to create p-r roc frame
    for cutoff in np.arange(0, 1 + res, res):

        # binarize decision to create confusion matrix values
        frame_[dname] = np.where(frame_[yhat] > cutoff , 1, 0)
        
        # calculate confusion matrix values
        tp = frame_[(frame_[dname] == pos) & (frame_[y] == pos)].shape[0]
        fp = frame_[(frame_[dname] == pos) & (frame_[y] == neg)].shape[0]
        tn = frame_[(frame_[dname] == neg) & (frame_[y] == neg)].shape[0]
        fn = frame_[(frame_[dname] == neg) & (frame_[y] == pos)].shape[0]

        # calculate precision, recall, and f1
        recall = (tp + eps)/((tp + fn) + eps)
        precision = (tp + eps)/((tp + fp) + eps)
        f1 = 2/((1/(recall + eps)) + (1/(precision + eps)))
        
        # add new values to frame
        prroc_frame = prroc_frame.append({'cutoff': cutoff,
                                          'recall': recall,
                                          'precision': precision,
                                          'f1': f1}, 
                                          ignore_index=True)
    
    # housekeeping
    del frame_
    
    return prroc_frame

# calculate and display recall and precision
prauc_frame = get_prauc(test_yhat, y, yhat)
prauc_frame.style.set_caption('Recall and Precision')

#### Find and display cutoff with maximum F1
For this model, the best F1 value is found at a probability cutoff of 0.25.

In [ ]:
gbm_cut = prauc_frame.loc[prauc_frame['f1'].idxmax(), 'cutoff'] # value associated w/ index of max. F1
print('Best F1 threshold: %.2f' % gbm_cut)

#### Plot P-R AUC with best F1 and cutoff
A P-R AUC curve is a curve formed by plotting precision and recall across different cutoffs. (P-R AUC curves are often a bit more robust to imbalanced target classes than ROC curves.) This curve shows some odd behavior for high precision and low recall, potentially indicating instability in model predictions, but is smooth and mostly well-behaved where the cutoff is selected. Generally a smooth tradeoff bewteen precision and recall is preferred as different cutoffs are considered.

In [ ]:
title_ = 'P-R Curve: Best F1 Threshold = ' + str(gbm_cut)
ax = prauc_frame.plot(x='recall', y='precision', kind='scatter', title=title_, xlim=[0,1])
_ = ax.axvline(gbm_cut, color='magenta')

#### Function to calculate confusion matrices
Once a cutoff has been established, a confusion matrix can be created. A confusion matrix is basic model assessment technique that shows how often a model is correct and incorrect, and how it is incorrect, e.g. type I (false positive) and type II (false negative) errors. 

In [ ]:
def get_confusion_matrix(frame, y, yhat, by=None, level=None, cutoff=0.5):

    """ Creates confusion matrix from pandas dataframe of y and yhat values, 
        can be sliced by a variable and level.
 
    Args:
        frame: Pandas dataframe of actual (y) and predicted (yhat) values.
        y: Name of actual value column.
        yhat: Name of predicted value column.
        by: By variable to slice frame before creating confusion matrix, 
            default None.
        level: Value of by variable to slice frame before creating confusion 
               matrix, default None.
        cutoff: Cutoff threshold for confusion matrix, default 0.5. 

    Returns:
        Confusion matrix as pandas dataframe. 
    """
    
    # determine levels of target (y) variable
    # sort for consistency
    level_list = list(frame[y].unique())
    level_list.sort(reverse=True)

    # init confusion matrix
    cm_frame = pd.DataFrame(columns=['actual: ' +  str(i) for i in level_list], 
                            index=['predicted: ' + str(i) for i in level_list])
    
    # don't destroy original data
    frame_ = frame.copy(deep=True)
    
    # convert numeric predictions to binary decisions using cutoff
    dname = 'd_' + str(y)
    frame_[dname] = np.where(frame_[yhat] > cutoff , 1, 0)
    
    # slice frame
    if (by is not None) & (level is not None):
        frame_ = frame_[frame[by] == level]
    
    # calculate size of each confusion matrix value
    for i, lev_i in enumerate(level_list):
        for j, lev_j in enumerate(level_list):
            cm_frame.iat[j, i] = frame_[(frame_[y] == lev_i) & 
                                        (frame_[dname] == lev_j)].shape[0]
            # i,j vs. j,i - nasty little bug updated 8/30/19
    
    return cm_frame
    
cm = get_confusion_matrix(test_yhat, y, yhat, cutoff=gbm_cut)
cm.style.set_caption('Confusion Matrix')

The confusion matrix shows that the GBM model is more accurate than not because the true positive and true negative cells contain the largest values by far. But the GBM model seems to make a large number of type I errors, or false positive predictions. False positives are a known disparity issue, because for complex reasons, many credit scoring and other models tend to overestimate the likelihood of non-reference groups - typically not white males - to default. The large amount of false positves could indicate that a lot of derserving people will not recieve loans that they could actually pay back according to this model. This is a potential pathology that should definitely be investigated further.

## 5. Basic Residual Analysis

#### Score test data and calculate logloss residuals
Because special functionality in XGBoost to find Shapley contributions to logloss will be used later, logloss residuals are used in several places in this notebook.

In [ ]:
# shortcut name
resid = 'r_DEFAULT_NEXT_MONTH' 

# calculate logloss residuals
test_yhat[resid] = -test_yhat[y]*np.log(test_yhat[yhat]) -\
                       (1 - test_yhat[y])*np.log(1 - test_yhat[yhat])   
    
# check that logloss is calculated correctly
# should match eval-logloss above
print('Mean logloss residual: %.6f' % test_yhat[resid].mean())

#### Plot global logloss residuals
Plotting residuals is almost always a good idea. You can see all of your model's predictions in two-dimensions!

In [ ]:
 # initialize figure
fig, ax_ = plt.subplots(figsize=(8, 8))         

# plot groups with appropriate color
color_list = ['royalblue', 'magenta'] 
c_idx = 0
groups = test_yhat.groupby(y) # define groups for levels of PAY_0
for name, group in groups:
    ax_.plot(group.p_DEFAULT_NEXT_MONTH, group.r_DEFAULT_NEXT_MONTH, 
             label=' '.join([y, str(name)]),
             marker='o', linestyle='', color=color_list[c_idx], alpha=0.3)
    c_idx += 1
    
# annotate plot
_ = plt.xlabel(yhat)
_ = plt.ylabel(resid)
_ = ax_.legend(loc=1)
_ = plt.title('Global Logloss Residuals')

Some high-magnitude outlying residuals are visible. Who are these customers? Why is the model so wrong about them? And are they somehow excerting undue influence on other predictions? The model could be retrained without these individuals and restested as a potentially remediation strategy. From a security standpoint it can be interesting to examine insider attacks and false negatives. These are people who where issued a loan, but did not pay it back. Are any of these employees, contractors, consultants, or their associates? Could they have poisoned the model's training data to receive the loan? Could they have changed the model's production scoring code to receive the loan?

#### Examine high logloss individuals

In [ ]:
# sort by residual and display
test_yhat_sorted = test_yhat.\
    sort_values(by=resid, ascending=False).\
    reset_index(drop=True)
    
test_yhat_sorted.head() # large positive residuals

High positive residuals seem to be caused by people with excellent payment behavior who suddenly default. 

#### Examine low logloss residuals

In [ ]:
test_yhat_sorted.tail()

The model seems to have the easiest time making correct decisions about customers with good payment behavior who sustain that behavior.

#### Residuals plotted by important variables: `PAY_0`
To take residual analysis farther, try plotting residuals by values of important variables or across bins of important variables. 

In [ ]:
# some seaborn configs
sns.set(font_scale=0.9)                                         # legible font size
sns.set_style('whitegrid', {'axes.grid': False})                # white background, no grid in plots
sns.set_palette(sns.color_palette(["#4169e1", "#ff00ff"]))      # consistent colors

# facet grid of residuals by PAY_0 
sorted_ = test_yhat.sort_values(by='PAY_0')                     # sort for better layout of by-groups
g = sns.FacetGrid(sorted_, col='PAY_0', hue=y, col_wrap=4)      # init grid
_ = g.map(plt.scatter, yhat, resid, alpha=0.4)                  # plot points
_ = g.add_legend(bbox_to_anchor=(0.82, 0.2))                    # legend

Here, a problem with the model is visible. For people whose most recent repayment status, i.e. `PAY_0` value, is good, `PAY_0 <= 1`, the model appears to struggle to predict default. When the model predicts this type of customer will not default, and they do, this results in large residuals. For people with late most recent payments, `PAY_0 > 1`, the model appears to struggle to predict on-time payment. When the model predicts people with late most recent payments will default, and then they don't, this also causes large residuals. *A mechanism for error* has been indentified: over-emphasis of a feature: `PAY_0 > 2`. These problems could be become worse if market conditions change, such as in a recession, where more people with good payment behavior could default suddenly. 

#### Residuals plotted by important variables: `SEX`
While not a frontline fairness tool, it can be interesting to examine residuals across demographic variables to get a sense of whether the model's errors are different across these groups. 

In [ ]:
# facet grid of residuals by SEX
sorted_ = test_yhat.sort_values(by='SEX')                # sort for better layout of by-groups
g = sns.FacetGrid(sorted_, col='SEX', hue=y, col_wrap=4) # init grid 
_ = g.map(plt.scatter, yhat, resid, alpha=0.4)           # plot points
_ = g.add_legend(bbox_to_anchor=(0.6, 0.5))              # legend

The residual profile is not dramatically different for men (`SEX = 1`) and women (`SEX = 2`). The largest magnitude residuals appear for men and for false positve predictions. This does not ensure the model is not discriminatory, but is just a potentially interesting method to augment results from a more rigorous fairness analysis, such as: https://github.com/jphall663/interpretable_machine_learning_with_python/blob/master/dia.ipynb. For instance the women with the highest false positive residuals, would in some sense, be potentially the most discriminated against by the model.

## 6. Disparate Error Analysis
The analysis in this section applies simple but numerous metrics to the accuracy and error of the model across the values of important or demographic variables to discover any potential blindspots in the model's predictions.

#### Represent error metrics as dictionary for use later
These are the metrics that will be examined across important and demographic variables. These metrics are based on the confusion matrix, but any other metrics would likely be useful as well.

In [ ]:
metric_dict = {

#### overall performance
'Prevalence': '(tp + fn) / (tp + tn +fp + fn)', # how much default actually happens for this group
#'Adverse Impact': '(tp + fp) / (tp + tn + fp + fn)', # how often the model predicted default for each group   
'Accuracy':       '(tp + tn) / (tp + tn + fp + fn)', # how often the model predicts default and non-default correctly for this group

#### predicting default will happen
# (correctly)
'True Positive Rate': 'tp / (tp + fn)',  # out of the people in the group *that did* default, how many the model predicted *correctly* would default              
'Precision':          'tp / (tp + fp)',  # out of the people in the group the model *predicted* would default, how many the model predicted *correctly* would default

#### predicting default won't happen
# (correctly)
'Specificity':              'tn / (tn + fp)', # out of the people in the group *that did not* default, how many the model predicted *correctly* would not default
'Negative Predicted Value': 'tn / (tn + fn)', # out of the people in the group the model *predicted* would not default, how many the model predicted *correctly* would not default  

#### analyzing errors - type I
# false accusations 
'False Positive Rate':  'fp / (tn + fp)', # out of the people in the group *that did not* default, how many the model predicted *incorrectly* would default
'False Discovery Rate': 'fp / (tp + fp)', # out of the people in the group the model *predicted* would default, how many the model predicted *incorrectly* would default

#### analyzing errors - type II
# costly ommisions
'False Negative Rate': 'fn / (tp + fn)', # out of the people in the group *that did* default, how many the model predicted *incorrectly* would not default
'False Omissions Rate':'fn / (tn + fn)'  # out of the people in the group the model *predicted* would not default, how many the model predicted *incorrectly* would not default
}    

#### Small utility functions used to calculate and display different types of errors 
A number of small, tightly coupled utiltiy functions are used to calculate and display the error metrics across the values of important or demographic variables.

In [ ]:
# small utility functions
# all tightly coupled to global names and data structures !!

def cm_exp_parser(expression):
    
    """ Translates abbreviated metric expressions from metric_dict 
        into executable Python statements.
    
    Arg: 
        expression: Error metric expression from metric_dict.
        
    Returns:
        Python statements based on predefined metrics in metric_dict.
        
    """
    
    # tp | fp       cm_dict[level].iat[0, 0] | cm_dict[level].iat[0, 1]
    # -------  ==>  --------------------------------------------
    # fn | tn       cm_dict[level].iat[1, 0] | cm_dict[level].iat[1, 1]

    expression = expression.replace('tp', '(cm_dict[level].iat[0, 0] + eps)')\
                           .replace('fp', '(cm_dict[level].iat[0, 1] + eps)')\
                           .replace('fn', '(cm_dict[level].iat[1, 0] + eps)')\
                           .replace('tn', '(cm_dict[level].iat[1, 1] + eps)')

    return expression

################################################################################

def get_cm_dict(name, cutoff):

    """ Loops through levels of named variable and calculates confusion 
        matrices per level; uses dynamically generated entities to reduce 
        code duplication. 
    
    Args:
        name: Name of variable for which to calculate confusion matrices.
        cutoff: Cutoff threshold for confusion matrices. 
    
    Returns:
        Dictionary of confusion matrices. 
    
    """

    levels = sorted(list(test_yhat[name].unique())) # get levels
    cm_dict = {} # init dict to store confusion matrices per level
    for level in levels: 
    
        # dynamically name confusion matrices by level
        # coerce to proper python names
        cm_name = '_' + str(level).replace('-', 'm') + '_cm' 
    
        # dynamically calculate confusion matrices by level
        code = cm_name + ''' = get_confusion_matrix(test_yhat,                              
                          y, 
                          yhat, 
                          by=name, 
                          level=level, 
                          cutoff=cutoff)'''
        exec(code)
        exec('cm_dict[level] = ' + cm_name) # store in dict
        
    return cm_dict

################################################################################

def get_metrics_frame(name): 

    """ Loops through levels of named variable and metrics to calculate each 
        error metric per each level of the variable; uses dynamically generated 
        entities to reduce code duplication.
    
    Arg:
        name: Name of variable for which to calculate error metrics.
    
    Return:
        Pandas DataFrame of error metrics.
        
    """
    
    levels = sorted(list(test_yhat[name].unique())) # get levels
    metrics_frame = pd.DataFrame(index=levels) # init Pandas frame for metrics
    eps = 1e-20 # for safe numerical operations

    # nested loop through:
    # - levels
    # - metrics 
    for level in levels:
        for metric in metric_dict.keys():
              
            # parse metric expressions into executable pandas statements
            expression = cm_exp_parser(metric_dict[metric])
        
            # dynamically evaluate metrics to avoid code duplication
            metrics_frame.loc[level, metric] = eval(expression)  

    # display results
    return metrics_frame


#### Display all error metrics across levels of `PAY_0`

In [ ]:
name = 'PAY_0'
cm_dict = get_cm_dict(name, gbm_cut) # get dict of confusion matrices

# print formatted error metrics frame: precision, title, colors
PAY_0_metrics = get_metrics_frame(name)
PAY_0_metrics['PAY_0'] = [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
PAY_0_metrics.set_index('PAY_0', inplace=True)
PAY_0_metrics.round(3).\
    style.set_caption('Error Metrics for ' + name).\
    background_gradient(cmap=sns.diverging_palette(-20, 260, n=7, as_cmap=True), 
                        axis=1)

As identified by basic residual analysis, it can be seen here that the model behaves extremely differently for customers with `PAY_0 <= 1` and `PAY_0 > 1`, likely due to sparsity of data in the `PAY_0 > 1` range. Again, this may be acceptable behavior today, but what if market conditions drift and these types of behaviors become more common and model mistakes become more costly in the future?

#### Display all error metrics across levels of `SEX`

In [ ]:
name = 'SEX'
cm_dict = get_cm_dict(name, gbm_cut) # get dict of confusion matrices

# print formatted error metrics frame: precision, title, colors
sex_metrics = get_metrics_frame(name)
sex_metrics['SEX'] = ['Male', 'Female']
sex_metrics.set_index('SEX', inplace=True)
sex_metrics.round(3).\
    style.set_caption('Error Metrics for ' + name).\
    background_gradient(cmap=sns.diverging_palette(-20, 260, n=7, as_cmap=True), 
                        axis=1)

Unlike `PAY_0`, different types of error and accuracy measures appear fairly aligned for men and women.

## 7. Benchmark Models

Benchmark models are an excellent model debugging tool. They can be used at training time to understand how a new model differs from an established, trusted model. They can also be used at scoring time to understand if a newer or more complex model is giving different predictions from a previously deployed trusted model or simpler model. If a prediction from a new model is too different from a prediction from a trusted model, this could be indicative of potential accuracy, fairness, or security problems.

#### Function for simple grid search across alpha for penalized GLM
Instead of using a pre-existing model, a stable penalized GLM will be used as a benchmark in this notebook.

In [ ]:
def glm_grid(X, y, train, valid):
    
    """ Wrapper function for penalized GLM with alpha and lambda search.
    
    :param X: List of inputs.
    :param y: Name of target variable.
    :param train: Name of training H2OFrame.
    :param valid: Name of validation H2OFrame.
    :return: Best H2Omodel from H2OGeneralizedLinearEstimator

    """
    
    alpha_opts = [0.01, 0.25, 0.5, 0.99] # always keep some L2
    
    # define search criteria
    # i.e. over alpha
    # lamda search handled by lambda_search param below
    hyper_parameters = {'alpha': alpha_opts} 

    # initialize grid search
    grid = H2OGridSearch(
        H2OGeneralizedLinearEstimator(
            family="binomial",
            lambda_search=True,
            seed=SEED),
        hyper_params=hyper_parameters)
    
    # run grid search
    grid.train(y=y,
               x=X, 
               training_frame=train,
               validation_frame=valid)

    # select best model from grid search
    return grid.get_grid()[0]

best_glm = glm_grid(X, y, h2o.H2OFrame(train), h2o.H2OFrame(test))
print('Best penalized GLM mean logloss residual: %.6f' % 
      best_glm.logloss(valid=True))
glm_cut = best_glm.F1(valid=True)[0][0] # get GLM cutoff to create decisions
print('At threshold: %.6f' % glm_cut)

#### Find rows where GLM benchmark model is right, but GBM is wrong
One interesting place to start comparing a more complex model to a simpler model is when the simple model is right and the complex model is wrong.

In [ ]:
# copy test data
test_yhat2 = test_yhat.copy(deep=True)

# create columns for gbm and glm preds
test_yhat2.rename(columns={yhat: 'p_gbm_DEFAULT_NEXT_MONTH'}, inplace=True)
test_yhat2['p_glm_DEFAULT_NEXT_MONTH'] = best_glm.\
    predict(h2o.H2OFrame(test))['p1'].\
    as_data_frame()

# create columns for gbm and glm decisions (i.e. apply cutoff)
test_yhat2['gbm_DECISION'] = 0
test_yhat2.loc[test_yhat2['p_gbm_DEFAULT_NEXT_MONTH'] > gbm_cut,
               'gbm_DECISION'] = 1
test_yhat2['glm_DECISION'] = 0
test_yhat2.loc[test_yhat2['p_glm_DEFAULT_NEXT_MONTH'] > glm_cut,
               'glm_DECISION'] = 1

# create columns for gbm and glm wrong decisions
test_yhat2['gbm_WRONG'] = 0
test_yhat2.loc[test_yhat2[y] != test_yhat2['gbm_DECISION'], 'gbm_WRONG'] = 1
test_yhat2['glm_WRONG'] = 0
test_yhat2.loc[test_yhat2[y] != test_yhat2['glm_DECISION'], 'glm_WRONG'] = 1

# create a subset of preds where gbm is wrong, but glm is right
gbm_wrong = test_yhat2.loc[(test_yhat2['gbm_WRONG'] == 1) & 
                           (test_yhat2['glm_WRONG'] == 0)]

gbm_wrong[X + [y, 'p_glm_DEFAULT_NEXT_MONTH', 'p_gbm_DEFAULT_NEXT_MONTH']].head()

Strangely, for the subset of customers where the more complex GBM model is wrong and the GLM model is right, the predictions of two models appear to be inversely correlated. 

#### Plot rows where GLM benchmark model is right, but GBM is wrong

In [ ]:
# custom legend
custom = [Line2D([0], [0], color='royalblue', lw=2),
          Line2D([0], [0], color='deeppink', lw=2),
          Line2D([0], [0], marker='o', color='w',
          markerfacecolor='orange', markersize=3)]

# init plot
fig, ax = plt.subplots(figsize=(7, 5)) 

# plot sorted actuals
# double index reset orders index by sort variable and
# brings index into frame for plotting
_ = gbm_wrong[[y,'p_gbm_DEFAULT_NEXT_MONTH']].\
            sort_values(by='p_gbm_DEFAULT_NEXT_MONTH').\
            reset_index(drop=True).\
            reset_index().\
            plot(kind='scatter', x='index', y=y, color='orange', s=3, ax=ax, 
                 legend=False)

# plot sorted gbm and glm preds 
_ = gbm_wrong[['p_glm_DEFAULT_NEXT_MONTH', 'p_gbm_DEFAULT_NEXT_MONTH']].\
            sort_values(by='p_gbm_DEFAULT_NEXT_MONTH').\
            reset_index(drop=True).\
            plot(ax=ax, legend=False, 
                 title='Ranked Predictions for Correct GLM and Incorrect GBM')

# annotate plot
_ = ax.legend(custom, ['p_glm_DEFAULT_NEXT_MONTH', 'p_gbm_DEFAULT_NEXT_MONTH', 
                       y])
_ = ax.set_xlabel('Ranked Row Index')

For a range of probabilities between ~0.2 and ~0.6 there exists a group of customers where a GLM model gives more correct predictions than the more complex GBM model. In the plot above, the yellow points represent the known target labels, the pink line is the sorted GBM model predictions, and the blue line is the GLM predictions for the same customers and target labels. For this group of people the GLM is obviously able to better represent some attribute of the customer's demographics or behaviors. Can the differences between this group of people and the rest of the customers be identified and leveraged to make better predictions?

#### Descriptive statistics for rows where GLM benchmark model is right, but GBM is wrong

In [ ]:
gbm_wrong.describe()

If this group of people can be isolated, either by descriptive statistics, or by more sophisticated means, the training process could be adapted to fix these errors or another model could be used at scoring time to create more accurate predictions. Even if a group cannot be isolated, the two different model predictions could potentially be blended in this range of predicted probabilities. 

## 8. Explanation of Residuals
A novel and very interesting application of post-hoc explanation techniques is in-depth investigation of model residuals. Below post-hoc explanation of predictions and residuals are used to further debug model predictions.

### 8.1 Shapley Values for Residuals and Predictions

#### Calculate Shapley values for logloss for each row
A fairly recent addition to the `shap` package is the ability to calculate Shapley contributions to logloss. From this potentially very time-consuming calculation, an accurate estimate of each variable's contribution to the model's error for each row can be known. This notebook is shipped with pre-calculated values, but you can calculate them too. It might take a few hours though.

In [ ]:
# init explainer object for access to explainer.expected_value() function below
# expected value is a function for loss, not a constant
explainer = shap.TreeExplainer(xgb_model, test_yhat[X], 
                               feature_dependence='independent',  # necessary for logloss 
                               model_output='logloss')            # necessary for logloss

# load if pre-calculated
if os.path.isfile('shap_error_values.csv'):

    shap_error_values = np.loadtxt('shap_error_values.csv', delimiter=',') # load
    print('Pre-calculated Shapley values loaded from disk.') # print confirmation
    
# else, calculate
else: 

    xgb_model.set_attr(objective='binary:logistic')
    shap_error_values = explainer.shap_values(test_yhat[X], y=test_yhat[y]) # long step (hours!)
    np.savetxt('shap_error_values.csv', shap_error_values, delimiter=',') # save for immediate reuse later

#### Calculate Shapley values for prediction for each row

In [ ]:
shap_values = xgb_model.predict(dtest, pred_contribs=True) 

#### Plot Shapley value summary for global importance measure in prediction and residuals

Once the local Shapley contributions to the predictions and logloss are known, they can be aggregated by their mean absolute value to create global importance metrics. 

In [ ]:
# init two pane plot
fig, (ax0, ax1) = plt.subplots(ncols=2, sharey=False, figsize=(7, 5))
plt.subplots_adjust(left=0.0, right=1.2, wspace=0.4)

# plot global abs mean Shapley contribs to predictions
global_shap_values_pd = pd.DataFrame(np.abs(shap_values[:, :-1]).mean(axis=0),
                                     index=X, columns=['Pred. Contribs.'])
_ = global_shap_values_pd.sort_values(by='Pred. Contribs.').\
        plot(kind='barh', ax=ax0, legend=False, color='royalblue',
             title='Global Shapley Contributions to Prediction')
_ = ax0.set_xlabel('mean(|SHAP Contrib.|)')

# plot global abs mean Shapley contribs to logloss
global_shap_error_values_pd = pd.DataFrame(np.abs(shap_error_values).mean(axis=0), 
                                           index=X, columns=['Loss Contribs.'])
_ = global_shap_error_values_pd.sort_values(by='Loss Contribs.').\
        plot(kind='barh', ax=ax1, legend=False, color='royalblue',
             title='Global Shapley Contributions to Logloss')
_ = ax1.set_xlabel('mean(|SHAP Contrib.|)')

Some features are more important to the logloss than to the predictions! These include `PAY_3`, `PAY_2`, and `BILL_AMT1`. The model could potentially be retrained without these features.

#### Mean Shapley contributions of high and low residual customers
Shapley contributions to predictions are reliably available at scoring time. It may be possible to understand what Shapley contributions look like for correct model decisions and for incorrect model decisions and take remediation steps in real-time before issuing model predictions.

In [ ]:
# init two pane plot
fig, (ax0, ax1) = plt.subplots(ncols=2, sharey=True)
plt.subplots_adjust(left=0.0, right=1.5, wspace=0.1)

# convert shap_values into Pandas DataFrame 
# merge residuals onto this frame
local_shap_values_pd = pd.DataFrame(shap_values[:, :-1], columns=X)
local_shap_values_pd[resid] = test_yhat[resid]

# plot mean shap values of 100 lowest residual customers 
_ = local_shap_values_pd.sort_values(by=resid).\
        reset_index(drop=True).\
        iloc[:1000, :-1].mean(axis=0).\
        plot(kind='barh', color='royalblue', ax=ax0, 
             title='Mean Low Residual Pred. Contribs.')
_ = ax0.set_xlabel('Pred. Contribs.')

# plot mean shap values of 100 highest residual customers
_ = local_shap_values_pd.sort_values(by=resid).\
        reset_index(drop=True).\
        iloc[-1000:, :-1].mean(axis=0).\
        plot(kind='barh', color='royalblue', ax=ax1,
            title='Mean High Residual Pred. Contribs.')
_ = ax1.set_xlabel('Pred. Contribs.')

Here large differences in average Shapley values are visible between low and high residual model predictions. On average, correct model decisions have negative Shapley values, while incorrect decisions have small magnitude Shapley values. If, when issuing a prediction, the Shapley values are low magnitude, the row could be diverted to another model or sent for human processing. Input data, particularly features with large global or local Shapley contributions to loss, could also be corrupted with missing values in hopes of attaining a better model prediction. 

#### Residuals vs. local Shapley prediction values in 2-D
Shapley contributions to predictions can also be plotted against residual values, in hopes of understanding if certain model behaviors lead to higher errors.

In [ ]:
# init three pane plot
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, sharey=False, figsize=(7, 5))
plt.subplots_adjust(left=0.0, right=2, wspace=0.4)

# plot shap vs. important variable PAY_0
_ = local_shap_values_pd.plot(kind='scatter', y=resid, x='PAY_0', color='royalblue',
                        title='PAY_0 Pred. Contribs. vs. ' + resid, ax=ax0)
_ = ax0.set_xlabel('PAY_0 Pred. Contribs.')

# plot shap vs. important variable PAY_3
_ = local_shap_values_pd.plot(kind='scatter', y=resid, x='PAY_3', color='royalblue',
                        title='PAY_3 Pred. Contribs. vs. ' + resid, ax=ax1)
_ = ax0.set_xlabel('PAY_3 Pred. Contribs.')

# plot shap vs. demographic variable SEX
_ = local_shap_values_pd.plot(kind='scatter', y=resid, x='SEX', color='royalblue',
                        title='SEX Pred. Contribs. vs. ' + resid, ax=ax2)
_ = ax2.set_xlabel('SEX Pred. Contribs.')

Here it appears there are no single Shapley contributions that can be associated with only high residuals. Every extremely high residual value also has low residual values for the same Shapley value. However, it can be seen that some Shapley values are only associated with low residuals, for instance `0 < PAY_0` Contribs. `< 0.5` or `PAY_3` Contribs. `< -0.2`. If trying to detect scoring problems in real-time, Shapley values in these ranges for these features could indicate that these predictions are likely to be correct. Also, in terms of local feature importance, the model is not treating `SEX` very differently -- a good sign from a fairness perspective but not a definitive test. From a security perspective, high residuals above low residuals are potentially interesting. These are individuals the model usually gets right, but for some reason they did not in this case. That could indicate manipulation of the training data or model scoring logic in rare cases.

#### Residuals vs. local Shapley prediction values in 3-D
If relationships between Shapley contributions and dependably high residuals cannot be found in two-dimensions, they may be visible in three dimensions when two features interact.

In [ ]:
# init 3-D plot
fig, ax = plt.subplots(figsize=(10, 8))
ax = plt.axes(projection='3d')

# z-axis: residuals values
_z = local_shap_values_pd[resid].values
ax.set_zlabel(resid)

# x-axis: PAY_2 values
_x = local_shap_values_pd['PAY_2'].values
ax.set_xlabel('\nPAY_2 Pred. Contribs.')

# y-axis: PAY_3 values
_y = local_shap_values_pd['PAY_3'].values
ax.set_ylabel('\nPAY_3 Pred. Contribs.')

# plot and annotate
_ = ax.scatter3D(_x, _y, _z, color='royalblue')
fig.suptitle('PAY_2 and PAY_3 Pred. Contribs vs. ' + resid) 
plt.tight_layout()

Here two features that are globally more important to logloss than to the model predictions are plotted against model residuals.  Like directly above, these plots can also be used to find potentially interesting areas of regions of purely high or low residuals, differences in model behavior across demographic segments, or strange patterns indicating possible manipulation of data or model logic.

#### Shapley contributions to logloss for a specific individual
Much like individual Shapley contributions to predictions, individual Shapley contributions to logloss residuals show exactly how much a variable contributes to the logloss for any individual in the training data.

In [ ]:
# select highest residual individual
row_id = test_yhat_sorted.loc[0, 'ID']
row = test_yhat[test_yhat['ID'] == row_id]

# match to their shap loss contribs
# create Pandas DataFrame
# display
s_df = pd.DataFrame(shap_error_values[row.index[0], :].reshape(23, 1), 
                    columns=['Loss Contribs.'], index=X)
s_df.sort_values(by='Loss Contribs.', inplace=True, ascending=False)
s_df

For this high-residual individual, `PAY_AMT1`, `LIMIT_BAL`, and `PAY_AMT2` have the largest positive impact on the model error.

#### Check Shapley values for local accuracy
Just to make sure Shapley is locally accurate, it can be shown that the Shapley contributions sum directly to the logloss for this individual.

In [ ]:
# should match residual value for row ... very close
print('Total Shapley contributions to logloss: %.6f' % 
        (s_df['Loss Contribs.'].sum() + explainer.expected_value(row[y].values[0])))

#### Plot Shapley contributions to logloss for a specific individual

In [ ]:
# plot barchart of top five loss contribs
ax = s_df.plot(kind='bar', 
                  title='Local Feature Contributions to Logloss\n',
                  color='royalblue',
                  legend=False)

# annotate
_ = ax.set_ylabel('Loss Contribs.')

#### Combine Shapley values with row values to explain logloss
Just like reason codes, or simple written explanations, can be generated for each model prediction. Reason codes can also be generated for logloss for each model prediction by combining the ranking provided by Shapley values and the individual row values. 

In [ ]:
row # helps understand reason codes

For this individual the top five characteristics contributing to the logloss residual are:


* Most recent payment is NT dollar 81,690 (high)
* Credit limit is NT dollar 500,000 (high)
* Second most recent payment is NT dollar 18,225 (high)
* Most recent repayment status is `PAID IN FULL` (very good)
* Third most recent repayment status is `PAID IN FULL` (very good)

All of the variables contributing to high logloss for this individual have values that would be considered positive attributes: large payments, high credit limit, and good repayment statuses. Essentially the model is surprised this person defaulted. This could mean that the model is too reliant on repayment statuses, as was seen by plotting residuals against `PAY_0` values, or it may also indicate the model requires more information to make good decisions, likely in the form of additional input variables.

In general, when examing local contributions to logloss, one might consider whether local contributions to predictions and errors are parsimonious or might indicate tampering with the data or model scoring logic or whether protected variables are weighing heavily in a model decision. Also, and although tedious and time-consuming today, one could use sensitivity analysis to understand how changing model inputs could affect model outcomes for different individuals.

### 8.2  Explain LogLoss Residuals with Decision Trees
A time-tested method of determining if a model is fundementally biased, either missing data or too simple in form, is to determine whether strong patterns exist in the model residuals. One way to determine if these patterns exist is to model the residuals themselves. When the model used is a decision tree or rule-based model, this opens up the possibility of automatically generating rules that indicate when or why a model might be wrong. 

#### Small utility functions used to generate decision trees
This set of small functions is used to train decision trees on logloss residuals and display those trees.

In [ ]:
# small utility functions
# all tightly coupled to global names and data structures !!

def train_cv_dt(model_id, frame):
                             
    """ Utility function to train decision trees.
    
    Args:
        model_id: h2o model identifier. 
        frame: Pandas DataFrame containing X and yhat on which to train 
               the decision trees.
               
    Returns:
        Path to saved model MOJO (Java scoring artifact), 
        model as h2o model object.
    
    """
        
    # initialize single tree model
    tree = H2ORandomForestEstimator(ntrees=1,          # use only one tree
                                    sample_rate=1,     # use all rows in that tree
                                    mtries=-2,         # use all columns in that tree's split search
                                    max_depth=4,       # shallow trees are easier to understand
                                    seed=SEED,         # set random seed for reproducibility
                                    nfolds=3,          # cross-validation for stability and ...
                                                       # only way to get metrics for 1 tree in h2o
                                    model_id=model_id) # gives MOJO artifact a recognizable name

    # train single tree model
    tree.train(x=X, y=resid, training_frame=h2o.H2OFrame(frame))

    # persist MOJO (compiled Java representation of trained model)
    # from which to generate plot of tree
    mojo_path = tree.download_mojo(path='.')
    print('Generated MOJO path:\n', mojo_path)
    
    return mojo_path, tree

################################################################################

# 
def get_gv(title, model_id, mojo_path):

    """ Utility function to generate graphviz dot file from h2o MOJO using 
        a subprocess.
    
    Args:
        title: Title for displayed decision tree.
        model_id: h2o model identifier. 
        mojo_path: Path to saved model MOJO (Java scoring artifact); 
                   generated by train_cv_dt function above. 
                   
    """
    
    # locate h2o jar
    hs = H2OLocalServer()
    h2o_jar_path = hs._find_jar()
    print('Discovered H2O jar path:\n', h2o_jar_path)

    # construct command line call to generate graphviz version of 
    # tree, see for more information: 
    # http://docs.h2o.ai/h2o/latest-stable/h2o-genmodel/javadoc/index.html
    gv_file_name = model_id + '.gv'
    gv_args = str('-cp ' + h2o_jar_path +
                  ' hex.genmodel.tools.PrintMojo --tree 0 -i '
                  + mojo_path + ' -o').split()
    gv_args.insert(0, 'java')
    gv_args.append(gv_file_name)
    if title is not None:
        gv_args = gv_args + ['--title', title]
    
    # call constructed command
    print()
    print('Calling external process ...')
    print(' '.join(gv_args))
    _ = subprocess.call(gv_args)

################################################################################

def get_png(model_id):

    """ Utility function to generate PNGs from .dots using a subprocess.
    
    Arg:
        model_id: h2o model identifier. 
    
    """
    
    gv_file_name = model_id + '.gv'
    
    # construct call to generate PNG from 
    # graphviz representation of the tree
    png_file_name = model_id + '.png'
    png_args = str('dot -Tpng ' + gv_file_name + ' -o ' + png_file_name)
    png_args = png_args.split()

    # call 
    print('Calling external process ...')
    print(' '.join(png_args))
    _ = subprocess.call(png_args)
    

#### Train decision trees

In [ ]:
# train trees
mojo_path0, tree0 = train_cv_dt('tree0', test_yhat[test_yhat[y] == 0])                                     
mojo_path1, tree1 = train_cv_dt('tree1', test_yhat[test_yhat[y] == 1])  

# shutdown h2o
h2o.cluster().shutdown()

#### Display error measures to ensure trustworthiness of models: `DEFAULT_NEXT_MONTH = 0` logloss residuals

In [ ]:
# cv metrics for y = 0 tree
tree0.cross_validation_metrics_summary().as_data_frame()

The residuals for non-default decisions can be fit relatively well with a single shallow tree and with stable errors across three folds.

#### Display error measures to ensure trustworthiness of tree models: `DEFAULT_NEXT_MONTH = 1` logloss residuals

In [ ]:
# cv metrics for y = 1 tree
tree1.cross_validation_metrics_summary().as_data_frame()

The residuals for default decisions can be fit relatively well with a single shallow tree and with stable errors across three folds.

#### Create graphviz `.dot` files for visualizing decision trees

In [ ]:
get_gv('LogLoss Residual Tree (' + y + '=0)', 'tree0', mojo_path0) # .dot for y=0 tree
get_gv('LogLoss Residual Tree (' + y + '=1)', 'tree1', mojo_path1) # .dot for y=1 tree

#### Convert `.dot` files into `.png`s

In [ ]:
get_png('tree0') # .png for y=0 tree
get_png('tree1') # .png for y=1 tree

#### Decision tree for `DEFAULT_NEXT_MONTH` = 0 logloss residuals

In [ ]:
display(Image(('tree0.png')))

This relatively accurate tree shows that strong patterns likely exist in the model residuals for non-default decisions. The highest residuals appear to occur when: `PAY_0 >= 1.5` and `PAY_5 >= 1` and `BILL_AMT2 < 18776.5` and `PAY_4 >= 1`. This scenario could be easily converted into a *model assertion* for use when scoring new data. When these conditions are present, the model is likely to be wrong and action can be taken such as scoring this individual with another model, diverting them to a human case worker, simply predicting the mean value of `DEFAULT_NEXT_MONTH`, or injecting missing values into their data in hopes of issuing a better prediction. 

Since these residuals relate to individuals being awarded a loan, another potentially important pattern to be aware of in the residuals would be if employees, contractors, or consultants were consistently being awarded loans, particularly by high-residual model decisions. Such a pattern could be indicative of data poisoning or tampering with model scoring logic. It seems possible that these tree rules could also be used to create adversarial examples for white-hat hacking purposes.

#### Decision tree for `DEFAULT_NEXT_MONTH` = 1 logloss residuals

In [ ]:
display(Image(('tree1.png')))

This relatively accurate tree shows that strong patterns likely also exist in the model residuals for default decisions. Rules of this tree could be used to create model assertions as well, and because the decisions of this tree result in denial of credit, any rules relating demographic variables to large residuals may be indicative of sociological bias in model predictions.

#### Conclusion

This notebook uses techniques related to residual analysis to: 

* Visualize residuals and learn about outlying, wrong predictions and to see that the trained model is too dependent on one input variable: `PAY_0`.

* Examine multiple types of errors across the levels of categorical variables and found that the model behaves poorly for `PAY_0 >= 2` but seems to have roughly similar error characteristics across men and women.

* Compare monotonic GBM predictions to simpler linear model predictions and discovered a group of customers for which the simpler linear model performs better than the GBM.

* Analyze residuals with post-hoc explainable AI techniques to understand:

  * Shapley contributions to predictions and to the GBM model's logloss, and to see that some variables are actually more important globally to the logloss than to the predictions.
  * Shapley prediction contributions and their relationships to the model residuals, and discovered that some Shapley contribution values are only associated with correct model decisions.
  * Local Shapley contributions to logloss and to view the exact local contributions of variables to logloss for a high-residual individual.
  * Summaries of residual behavior through decision trees and potentially to create model assertions or adversarial examples automatically.

Once these kinds of model bugs have been identified, new options exist to remediate them during either training or scoring. This model was found to be too dependent on `PAY_0` and there was remaining signal in the model residuals to train accurate decision trees. It seems there is additional information that would lessen this model's over-emphasis of `PAY_0` and perhaps allow the constrained GBM to make more accurate decisions in general. Local contributions to logloss were also calculated. Is it possible to retrain the model with this information in some kind of novel row and column boosting scheme? Can the Shapley prediction contributions and the decision tree rules for residuals be used to create model assertions automatically? More importantly than this one toy case, can these strategies be applied to find accuracy, fairness, or security bugs in different types of models?